**Author:** Artyom Pak  
**Date:** 07/30/2021  
**Email:** artyomvp1@gmail.com

# Introduction
In this case study I will be analyzing video game sales (1980-2017). During the whole process I am going to use the Python programming language and demonstrate the skill working with data sets and visualizations. 

# Stage 1: Ask questions and determine tasks
#### The business tasks:
* Exploration and analysis
* Determine most successful companies, gaming genres, and platforms

# Stage 2: Preparation
**Data:** [click here](https://www.kaggle.com/gregorut/videogamesales)<br>
**License:** [click here](https://github.com/GregorUT/vgchartzScrape/blob/master/LICENSE)<br>
**Description**: This data set contains the information about video game sales above 100 000 copies. The data includes mostly the information about sales divided by regions.<br> However the information contains the data **only up to 2017**.<br>

#### Fields include
**Rank** - *Ranking of overall sales*<br>
**Name** - *The games name*<br>
**Platform** - *Platform of the games release*<br>
**Year** - *Year of the game's release*<br>
**Genre** - *Genre of the game*<br>
**Publisher** - *Publisher of the game*<br>
**NA_Sales** - *Sales in North America (in millions)*<br>
**EU_Sales** - *Sales in Europe (in millions)*<br>
**JP_Sales** - *Sales in Japan (in millions)*<br>
**Other_Sales** - *Sales in the rest of the world (in millions)*<br>
**Global_Sales** - *Total worldwide sales*<br>

# Stage 3: Process the data
The observation and analysis will be conducted using Python 3 (libraries `pandas`, `numpy` and `matplotlib`).

### 3.1 Importing libraries and data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from wordcloud import WordCloud

In [ ]:
games = pd.read_csv('../input/videogamesales/vgsales.csv')
games.head(5)

### 3.2. Observing the data set
According to the brief information below, the `Year` column contains a `float` data type. Also, the number of non-null values in the`Year` and `Publisher` columns is different next to the other columns. Duplicated values have not been found.

In [ ]:
# Common information about the data set and attributes
games.info()

In [ ]:
# Empty values check
games.isna().any()

In [ ]:
# Duplicated rows check
games.duplicated().any()

### 3.3 Cleaning 
Since I am not able to get the information about the empty cells in the `Year` column, these rows will be dropped. The empty `Publisher` cells will be replaced with `Unknown` to minimize the cleaning impact to the analysis.

In [ ]:
# Drop NA-values in YEAR
games.dropna(subset=['Year'], inplace=True)

# Handling NA-values in the Publisher
games.loc[games['Publisher'].isna() == True, ['Publisher']] = 'Unknown'

# Fixing 'Year' format
games['Year'] = games['Year'].apply(int)

# Stage 4: Analysis and visualization

### 4.1 Aggregated summary of top 100 games.
Having collected the summary we can clearly see that sales in the Northern America region are significantly bigger.

In [ ]:
# Sorting and extracting top 100
hundred = games.sort_values('Global_Sales', ascending=False).head(100)

# Changing float format
pd.options.display.float_format = "{:.2f}".format

# Aggregated statistics
hundred[['NA_Sales','EU_Sales', 'JP_Sales', 'Other_Sales','Global_Sales']].describe()

### 4.2 Global sales per year
As we see on the diagram below, the growth started in the 2000 and reached its peak a decade after. The growth is related to the popularity of game consoles starting in the beginning of the 21th century, whereas the downhill after 2009-2010 is coinciding with the [boom of the mobile gaming](https://www.statista.com/statistics/292507/mobile-contents-market-value-north-america/) industry. 

In [ ]:
df_year = games.groupby('Year').sum().reset_index(drop=False)

# Plot parameters 
figsize = [20, 7]
x = df_year['Year']
y = df_year['Global_Sales']
width = 0.8
color = 'mediumpurple'

# Bar chart
plt.figure(figsize=figsize)
bars = plt.bar(x, y, 
               width=width, 
               color=color)

# Non-data objects
plt.title('Global Sales per Year', fontdict={'fontweight':'bold', 'fontsize':20})    # A bit of hardcoding
plt.xlabel('Year', fontdict={'fontsize':18, 'fontweight':'bold'})
plt.ylabel('Dollars ($M)', fontdict={'fontsize':18, 'fontweight':'bold'})
plt.xticks(df_year['Year'], rotation = 90)

plt.show()

### 4.3 Popularity of genres
The bar chart below represents the popularity of genres. [Action](https://en.wikipedia.org/wiki/Action_game), [sports](https://en.wikipedia.org/wiki/Sports_video_game), and [shooter](https://en.wikipedia.org/wiki/Shooter_game) are on the top and getting together they make up almost the third part of the whole. On the other hand adventure, strategy, and puzzles show unpopularity of genre. 

A video game genre is a specific category of games related by similar gameplay characteristics. Video game genres are not usually defined by the setting or story of the game or its medium of play, but by the way the player interacts with the game. Basically, genre is a classification assigned to a video game based primarily on its gameplay (type of interaction) rather than visual or narrative features.

In [ ]:
# Summary by genre
df_genre = games.groupby('Genre').sum().reset_index(drop=False)

# Plot parameters
figsize = [12, 12]
x = [i for i in df_genre['Global_Sales']]
labels = [i for i in df_genre['Genre']]
colors = ['yellowgreen','pink','gold','lightskyblue','seagreen','lightcoral','lightblue',
          'grey', 'purple', 'orange','plum','violet','magenta','cyan']
explode = [0.05 if df_genre['Genre'][i] in ['Action', 'Sports', 'Shooter'] else    # List comprehension
           0.08 if df_genre['Genre'][i] in ['Strategy', 'Puzzle', 'Adventure'] else 
           0.01 for i in range(len(df_genre['Genre']))]
autopct = '%.1f %%'
textprops = {'fontsize': 15}

# Pie chart
plt.figure(figsize=figsize)
plt.pie(x, 
        labels=labels, 
        colors=colors,
        autopct=autopct, 
        explode=explode,
        textprops=textprops)

# Make a hole, so it's a donut
my_circle=plt.Circle((0,0), 0.45, color='white')    # (0,0) - coordinated of the center, 0.45 - diameter
p=plt.gcf()
p.gca().add_artist(my_circle)

# Non-data objects
plt.title('Genre Popularity', fontdict={'fontweight':'bold', 'fontsize':20})

plt.show()

### 4.4 Platform
According to the data, [PlayStation 2](https://en.wikipedia.org/wiki/PlayStation_2), [Xbox 360](https://en.wikipedia.org/wiki/Xbox_360), and [PlayStation 3](https://en.wikipedia.org/wiki/PlayStation_3) have the most successful **game sales** rates globally.  
Game consoles has been popular for a few decades because of the advantages that consoles can provide over PCs (easy to use, cost less, no need to upgrade, etc.)

In [ ]:
# Grouping sales by platform
platform = games[['Platform', 'NA_Sales', 'EU_Sales', 'JP_Sales', 'Global_Sales']]
platform = platform.groupby('Platform').sum().sort_values('Global_Sales', ascending=False).reset_index(drop=False).head(10)
platform.style.set_properties(**{'background-color': 'turquoise'}, subset=['Global_Sales'])    # Highlight c column

### 4.5 Publishers

Following, I would like to emphasize three most successful companies and genres of video games they are focused on.  
[Nintendo](https://www.macrotrends.net/stocks/charts/NTDOY/nintendo/revenue), [EA](https://www.macrotrends.net/stocks/charts/EA/electronic-arts/revenue), and [Activision](https://www.macrotrends.net/stocks/charts/ATVI/activision-blizzard/revenue) have been showing a decent level of revenue over the years.

In [ ]:
# Grouping sales by genre
publishers = games[['Publisher', 'NA_Sales', 'EU_Sales', 'JP_Sales', 'Global_Sales']]
publishers = publishers.groupby('Publisher').sum().sort_values('Global_Sales', ascending=False).reset_index(drop=False).head(10)
publishers.style.set_properties(**{'background-color': 'lightgreen'}, subset=['Global_Sales'])    # Highlight a column

Although all of them spread their efforts to all video game markets, having visualized the data, it became clear that the companies have a tendency to focus on particular genres. For instance, the bar chart below shows that Nintendo makes the most of their revenue from platform games, whereas EA (well known for its sport simulator games) and Activision are aimed on sport games and shooters accordingly. 

In [ ]:
# Subsetd based on publisher
nd = games.loc[games['Publisher'] == 'Nintendo', ['Genre', 'Global_Sales']].groupby('Genre').sum().reset_index(drop=False)
ea = games.loc[games['Publisher'] == 'Electronic Arts', ['Genre', 'Global_Sales']].groupby('Genre').sum().reset_index(drop=False)
av = games.loc[games['Publisher'] == 'Activision', ['Genre', 'Global_Sales']].groupby('Genre').sum().reset_index(drop=False)

# Axis parameters
labels = [i for i in nd['Genre']]
y1 = np.array([i for i in nd['Global_Sales']])  # Has to be an array!
y2 = np.array([j for j in ea['Global_Sales']])
y3 = np.array([k for k in av['Global_Sales']])

# Bar chart (clustered)
plt.figure(figsize=(12, 10))
plt.bar(labels, y1, color='#d67ed0')
plt.bar(labels, y2, bottom=y1, color='#e6ad12')
plt.bar(labels, y3, bottom=y1+y2, color='#13c5ed')  # bottom - stacking

# Non-data objects
plt.title('Top Publishers and Genre', fontdict={'fontweight':'bold', 'fontsize':20})
plt.xlabel('Genre', fontdict={'fontsize':18, 'fontweight':'bold'})
plt.ylabel('Dollars ($M)', fontdict={'fontsize':18, 'fontweight':'bold'})
plt.xticks(rotation=90)
plt.legend(['Nintendo', 'Electronic Arts', 'Activision'], fontsize=18)

plt.show()

### Summary
* The most profitable region - **North America**
* Popular video game genres are **action**, **sport**, **shooter**
* Platforms with the biggest number of video games released: **PS2**, **Xbox 360**, **PS3**
* The most successful video game publishers: **Nintendo Co. Ltd.**, **Electronic Arts Inc.**, **Activision Publishing, Inc.**

# Recap
Computer games have been around for decades, providing entertainment for children and adults as well. Video games are a billion-dollar business and have been for many years. The video game industry has boomed in recent years because of the variety of ways to play games. As technology continues to improve, so do video games.

In [ ]:
# Creating word cloud
text = ' '.join(games['Publisher'].unique())
wordcloud = WordCloud(max_font_size=100, background_color='gray').generate(text)

plt.figure(figsize=(17, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')

plt.show()